In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import color, img_as_float, restoration
from numpy.fft import fft2, ifft2, fftshift, ifftshift

# --- Load Image ---
img = cv2.imread("/content/Hst12.jpg")
astro = color.rgb2gray(img_as_float(img))

# --- Define PSF (example: Gaussian 5x5) ---
def gaussian_psf(size=5, sigma=1.0):
    ax = np.linspace(-(size-1)/2, (size-1)/2, size)
    xx, yy = np.meshgrid(ax, ax)
    psf = np.exp(-(xx**2 + yy**2) / (2*sigma**2))
    psf /= psf.sum()
    return psf

psf = gaussian_psf(size=7, sigma=1.2)

def wiener_filter(img, kernel, K):
	kernel /= np.sum(kernel)
	dummy = np.copy(img)
	dummy = fft2(dummy)
	kernel = fft2(kernel, s = img.shape)
	kernel = np.conj(kernel) / (np.abs(kernel) ** 2 + K)
	dummy = dummy * kernel
	dummy = np.abs(ifft2(dummy))
	return dummy

# --- Apply Wiener filter ---
astro_wiener = wiener_filter(astro, psf, K=0.5)  # tune K (0.001–0.02)

# --- Richardson-Lucy deconvolution ---
deconv_rl = restoration.richardson_lucy(astro_wiener, psf, num_iter=30, clip=True)

# --- Display Results ---
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(astro, cmap='gray')
plt.title("Original Image")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(deconv_rl, cmap='gray')
plt.title("After Wiener + Richardson-Lucy Deconvolution")
plt.axis('off')

plt.tight_layout()
plt.show()

# --- Save outputs ---
cv2.imwrite("Wiener_filtered.png", (astro_wiener * 255).astype(np.uint8))
cv2.imwrite("Wiener_RL.png", (deconv_rl * 255).astype(np.uint8))